In [1]:
import sys
sys.path.append('./scripts')

import numpy as np
import pandas as pd
import os
import re

from mp_api.client import MPRester

from data_featurization import (load_data, load_cif_structure, load_ordered_cif_structure, remove_charges, simplify_structure, 
remove_li_from_mixed_sites, construct_megnet_graph, get_megnet_feature, get_megnet_composition_feature, find_duplicate_structures, 
get_space_group, get_el2md_mapper, get_formula, get_normalized_formula)

from tqdm.auto import tqdm as tqdm_pandas
import warnings

warnings.filterwarnings("ignore")
tqdm_pandas.pandas()

In [2]:
def check_in_icsd(row):
    if isinstance(row['database_IDs'], dict) and 'icsd' in row['database_IDs']:
        return False
    else:
        return True

### Define paths for relevant directories

ICSD cifs are not public and so not included in this repository.

In [3]:
icsd_database_path = r".\data\li_containing_materials_icsd_03272024.csv"
labelled_conductivity_database_path = r".\data\ionic_conductivity_database_11022023.csv"
icsd_li_cifs_path = r"C:\Users\mchaf\Desktop\icsd_li_cifs"

pretrained_model_path_eform = r".\megnet\mvl_models\mp-2019.4.1\formation_energy.hdf5"
pretrained_model_path_bg = r".\megnet\mvl_models\mp-2018.6.1\band_gap_regression.hdf5"
pretrained_disordered_model_path_eform = r".\mp-2019_4_1_formation_energy_disordered"
pretrained_disordered_model_path_bg = ".\mp-2018_6_1_band_gap_regression_disordered"

### Load and filter ICSD database entries

In [4]:
icsd_database = pd.read_csv(icsd_database_path)
print(f"There are currently {len(icsd_database)} Li-containing materials in the ICSD as of 03/27/2024.")

There are currently 11295 Li-containing materials in the ICSD as of 03/27/2024.


### Load structures and check how many are disordered

In [5]:
icsd_database['structure'] = icsd_database['icsd_collectioncode'].progress_apply(load_cif_structure, cif_dir=icsd_li_cifs_path)
incorrectly_formatted_cifs = icsd_database['structure'].isna().sum()
print(f"{incorrectly_formatted_cifs} cif files could not be read properly")

  0%|          | 0/11295 [00:00<?, ?it/s]

Invalid CIF file with no structures!
The structure could not be loaded for ICSD entry 78313
Invalid CIF file with no structures!
The structure could not be loaded for ICSD entry 78314
Invalid CIF file with no structures!
The structure could not be loaded for ICSD entry 185529
Invalid CIF file with no structures!
The structure could not be loaded for ICSD entry 185530
Invalid CIF file with no structures!
The structure could not be loaded for ICSD entry 96650
Invalid CIF file with no structures!
The structure could not be loaded for ICSD entry 96649
Invalid CIF file with no structures!
The structure could not be loaded for ICSD entry 96652
Invalid CIF file with no structures!
The structure could not be loaded for ICSD entry 96651
Invalid CIF file with no structures!
The structure could not be loaded for ICSD entry 96654
Invalid CIF file with no structures!
The structure could not be loaded for ICSD entry 96653
Invalid CIF file with no structures!
The structure could not be loaded for ICS

In [6]:
disordered_count = 0
ordered_count = 0
for struc in icsd_database['structure'].to_list():
    if struc:
        if struc.is_ordered:
            ordered_count += 1
        else:
            disordered_count += 1

# There are 12 Li-containing cifs that cannot be read by Pymatgen due to improperly formatting. These are all disordered.
disordered_count += 12
print(f"Of the {len(icsd_database)} Li-containing materials in the ICSD as of 03/27/2024, {disordered_count} contain site disorder")

Of the 11295 Li-containing materials in the ICSD as of 03/27/2024, 6860 contain site disorder


In [13]:
labelled_conductivity_database = pd.read_csv(labelled_conductivity_database_path)
labelled_collectioncodes = set(labelled_conductivity_database['icsd_collectioncode'])
icsd_database_unlabelled = icsd_database[~icsd_database['icsd_collectioncode'].isin(labelled_collectioncodes)]
print(f"There are {len(icsd_database_unlabelled)} that do not have associated ionic conductivity values.")

There are 10725 that do not have associated ionic conductivity values.


### Perform structure simplification

In [8]:
icsd_database_unlabelled['structure_simplified'] = icsd_database_unlabelled['structure'].progress_apply(simplify_structure)

  0%|          | 0/10725 [00:00<?, ?it/s]

### Construct MEGNet graphs using linear combination of embeddings to represent disordered sites and featurize data using MEGNet Site describer

In [ ]:
cg_disordered = CrystalGraphDisordered(bond_converter=GaussianDistance(np.linspace(0, 6, 100), 0.5), cutoff=5)
pretrained_model = MEGNetModel.from_file(pretrained_model_path_eform)
weights = pretrained_model.get_weights()
elemental_embeddings = weights[0]

structure_columns = ['structure', 'structure_simplified']
feature_columns = []
for structure_type in structure_columns:
    # Define column name for the corresponding MEGNet graph representation
    graph_column_name = "megnet_graph_" + structure_type
    icsd_database_rt_unlabelled[graph_column_name] = icsd_database_rt_unlabelled[structure_type].progress_apply(construct_megnet_graph, graph_converter=cg_disordered, embeddings=elemental_embeddings)
    # Iterate over different levels of MEGNet site describers
    for i in range(1, 4):
        # Initialize MEGNetSite describer for the current level
        megnet_describer = MEGNetSite(level=i, name=pretrained_disordered_model_path_eform)
        feature_column_name = structure_type + f"_megnet_site_feature_level_{i}_2019_4_1_formation_energy"
        icsd_database_rt_unlabelled[feature_column_name] = icsd_database_rt_unlabelled[graph_column_name].progress_apply(get_megnet_feature, describer=megnet_describer)
icsd_database_rt_unlabelled.to_pickle(".\data\icsd_database_unlabelled_featurized.pkl")

In [9]:
icsd_database_unlabelled_featurized = pd.read_pickle(".\data\icsd_database_unlabelled_featurized.pkl")
icsd_database_unlabelled_featurized = icsd_database_unlabelled_featurized[icsd_database_unlabelled_featurized['structure'].notna()]
icsd_database_unlabelled_featurized['normalized_formula'] = icsd_database_unlabelled_featurized['structure'].progress_apply(get_normalized_formula)
icsd_database_unlabelled_featurized['formula'] = icsd_database_unlabelled_featurized['structure'].progress_apply(get_formula)

  0%|          | 0/10714 [00:00<?, ?it/s]

  0%|          | 0/10714 [00:00<?, ?it/s]

### Retrieve bandgap from Materials Project

In [19]:
with MPRester("***REMOVED***") as mpr:
    docs = mpr.summary.search(elements=["Li"], fields=['material_id', 'composition', 'composition_reduced', 'formula_pretty', 'structure', 'band_gap', 'energy_above_hull', 'theoretical', 'database_IDs'])
docs = [dict(i) for i in docs]
mp_df = pd.DataFrame(docs)
mp_icsd_band_gap_dict = {}
for index, row in mp_df.iterrows():
    if 'icsd' in row['database_IDs']:
        icsd_values = row['database_IDs']['icsd']
        band_gap = row['band_gap']
        for icsd in icsd_values:
            icsd_numerical = int(icsd.split('-')[1])
            mp_icsd_band_gap_dict[icsd_numerical] = band_gap

icsd_database_unlabelled_featurized['band_gap_mp'] = icsd_database_unlabelled_featurized['icsd_collectioncode'].progress_apply(extract_band_gap_mp, mp_icsd_band_gap_dict=mp_icsd_band_gap_dict)

Retrieving SummaryDoc documents:   0%|          | 0/21686 [00:00<?, ?it/s]

  0%|          | 0/10714 [00:00<?, ?it/s]

### Predict bandgap with MEGNet Pretrained model

In [20]:
cg_disordered = CrystalGraphDisordered(bond_converter=GaussianDistance(np.linspace(0, 6, 100), 0.5), cutoff=5)
pretrained_model = MEGNetModel.from_file(pretrained_model_path_bg)
weights = pretrained_model.get_weights()
elemental_embeddings = weights[0]
pretrained_disordered_model_bg = MEGNetModel.from_file(pretrained_disordered_model_path_bg)
icsd_database_unlabelled_featurized['megnet_graph_structure_band_regression'] = icsd_database_unlabelled_featurized['structure'].progress_apply(construct_megnet_graph, graph_converter=cg_disordered, embeddings=elemental_embeddings)
icsd_database_unlabelled_featurized['band_gap_prediction_megnet'] = icsd_database_unlabelled_featurized['megnet_graph_structure_band_regression'].progress_apply(predict_band_gap, model=pretrained_disordered_model_bg)
icsd_database_unlabelled_featurized.to_pickle(".\data\icsd_database_unlabelled_featurized_w_bg.pkl")

  0%|          | 0/10714 [00:00<?, ?it/s]

  0%|          | 0/10714 [00:00<?, ?it/s]

## Featurize Li-containing Materials in Materials Project 

In [16]:
mp_database = mp_df[mp_df.apply(check_in_icsd, axis=1)]
mp_database = mp_database[['material_id', 'formula_pretty', 'structure', 'energy_above_hull', 'database_IDs', 'theoretical', 'band_gap']]
mp_database['normalized_formula'] = mp_database['structure'].progress_apply(get_normalized_formula)
mp_database['formula'] = mp_database['structure'].progress_apply(get_formula)
mp_database['structure'] = mp_database['structure'].progress_apply(add_oxidation_state)
mp_database['structure_simplified'] = mp_database['structure'].progress_apply(simplify_structure)


  0%|          | 0/19451 [00:00<?, ?it/s]

  0%|          | 0/19451 [00:00<?, ?it/s]